In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.applications import MobileNetV2
import matplotlib.pyplot as plt
import os

In [4]:
BATCH_SIZE = 32
IMAGE_SIZE = 256

In [5]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "D:/IIT Docs/2nd Year/Semester 1/CM 2603 - Data Science Project/Data/Dataset/Potato/color",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 2152 files belonging to 3 classes.


In [6]:
# Preprocess dataset
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
dataset = dataset.map(lambda x, y: (preprocess_input(x), y))

In [7]:
# Split dataset into train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

In [8]:
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

In [9]:
# Create base model (MobileNetV2)
base_model = MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                         include_top=False,
                         weights='imagenet')

In [10]:
# Freeze base model
base_model.trainable = False

In [11]:
# Add classification head
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(3, activation='softmax')

In [12]:
# Build the model
inputs = tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
x = base_model(inputs, training=False)
x = global_average_layer(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [13]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [14]:
# Train the model
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=8)

Epoch 1/8
54/54 [==============================] - 25s 386ms/step - loss: 0.3506 - accuracy: 0.8860 - val_loss: 0.2017 - val_accuracy: 0.9612
Epoch 2/8
54/54 [==============================] - 20s 358ms/step - loss: 0.1343 - accuracy: 0.9659 - val_loss: 0.1360 - val_accuracy: 0.9569
Epoch 3/8
54/54 [==============================] - 19s 355ms/step - loss: 0.1001 - accuracy: 0.9745 - val_loss: 0.1197 - val_accuracy: 0.9655
Epoch 4/8
54/54 [==============================] - 19s 353ms/step - loss: 0.0795 - accuracy: 0.9769 - val_loss: 0.0972 - val_accuracy: 0.9655
Epoch 5/8
54/54 [==============================] - 19s 355ms/step - loss: 0.0652 - accuracy: 0.9821 - val_loss: 0.0916 - val_accuracy: 0.9698
Epoch 6/8
54/54 [==============================] - 19s 353ms/step - loss: 0.0523 - accuracy: 0.9878 - val_loss: 0.0796 - val_accuracy: 0.9741
Epoch 7/8
54/54 [==============================] - 19s 351ms/step - loss: 0.0458 - accuracy: 0.9907 - val_loss: 0.0686 - val_accuracy: 0.9698
Epoch 

In [15]:
# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')
print("Accuracy: ", accuracy * 100, "%")

8/8 [==============================] - 4s 290ms/step - loss: 0.0562 - accuracy: 0.9883
Test accuracy: 0.98828125
Accuracy:  98.828125 %


In [16]:
# Save the model
model_version = max([int(i) for i in os.listdir("../mobilenet_models") + [0]]) + 1
model.save(f"../mobilenet_models/{model_version}")

INFO:tensorflow:Assets written to: ../mobilenet_models/2\assets


INFO:tensorflow:Assets written to: ../mobilenet_models/2\assets


In [17]:
model.save("potato_mobilenetv2.h5")

D:\IIT Docs\2nd Year\Semester 1\CM 2603 - Data Science Project\Project\SampleFlask\PotatoDiseaseDetectionFlaskJinja\.venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("potato_mobilenetv2.tflite", 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpn1ikpmsv\assets


INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpn1ikpmsv\assets
